In [51]:
import numpy as np
import pandas as pd 

In [52]:
import matplotlib.pyplot as plt
import seaborn as sns

In [53]:
from modules import TrainData, one_hot ,position_transform, wakati, meishi,FastText_Vectrizer,rename_section,exam_preprosses,to_vec

In [54]:
# データの修正

In [95]:
df = pd.read_csv('DATA20200401_20220331(修正後).csv', encoding='cp932', header=None)

In [96]:
# ラベルの間違い修正
df[3].unique()

array(['A1', 'A2', 'AB1', 'AB3', 'AB4', 'AB5', 'AB6', 'B1', 'B2', 'BL1',
       'C1', 'C2', 'C３', 'C3', 'C3.', 'DW', 'H1', 'H10', 'H12', 'H13',
       'H13 ', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19', 'H2', 'H20',
       'H3', 'H4', 'H5', 'H6', 'H6\n', 'H7', 'H７', 'H7.', 'H8', 'H８',
       'H9', 'HS', 'K1', 'K2', 'L1', 'L2', 'L3', 'L4', 'L5', 'LI1',
       'LI1\n', 'LI2', 'LI3', 'LI3\n', 'LI4', 'N1', 'P1', 'PL1', 'R1',
       'R2', 'S1', 'S1\n', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S9', 'U1',
       'U2', 'U3', 'U4', 'U5', nan], dtype=object)

In [97]:
df[3]=df[3].replace(['C３', 'C3', 'C3.'], 'C3')
df[3]=df[3].replace(['H13', 'H13 '], 'H13')
df[3]=df[3].replace(['H6', 'H6\n'], 'H6')
df[3]=df[3].replace(['H7', 'H７', 'H7.'], 'H7')
df[3]=df[3].replace(['H8', 'H８'], 'H8')
df[3]=df[3].replace(['LI3\n'], 'LI3')
df[3]=df[3].replace(['S1', 'S1\n'], 'S1')
df[3]=df[3].replace(['LI1','LI1\n'], 'LI1')

In [98]:
df[df[3]=='H14']

,0,1,2,3,4,5,6,7,8
4063,11,女,耳鼻,H14,頭部(脳),嗅球プロトコル。先天性嗅覚障害です ３Tでお願いします。,くも膜のう胞,依頼通りに嗅球プロトコル,単純


In [99]:
df[2] = df[2].replace('H1', '脳外')

In [100]:
df[3] = df[3].replace('H14', 'N1')

In [101]:
df[df.isnull().any(axis=1)]

,0,1,2,3,4,5,6,7,8
3,55,男,血内,A2,下肢血管：静脈（単純）,上記症例。 左下肢血栓症の疑い。本人の訴えは、大腿部から下腿にかけてです。 本人の強い希望に...,多発性骨髄腫,NaN,単純
128,80,女,放科,AB3,骨盤部,病診：いしやま内科整形外科よりご紹介。腹部不快感で受診。腹部エコーで卵巣腫大の疑い。精査目的。,卵巣腫瘍の疑い,NaN,単純
206,32,女,脳内,AB3,婦人科下腹部,NMDA受容体脳炎疑いで救急搬送。下腹部CTで奇形 腫の疑いあり。MRIにて精査お願いします。,卵巣奇形腫の疑い,NaN,単純
327,45,女,放科,AB3,婦人科下腹部,病診 不整出血にて受診、自然に改善したが、その際の診察エコーにて左卵巣腫瘍を疑う（卵管血腫？...,卵巣腫瘍,NaN,単純
375,52,女,母女,AB3,婦人科下腹部,急性腹症で救急受診した患者。 骨盤内に充実部分を伴う嚢胞性腫瘤および腹水あり。 精査目的です。,卵巣癌の疑い,NaN,単純＋造影Gd
...,...,...,...,...,...,...,...,...,...
18754,84,女,救急,S5,全脊椎(単純)Sagのみ,1ヶ月ほど前より背部痛あり。3/23CTで複数の圧 迫骨折指摘を指摘された。何とか歩行できて...,胸腰椎圧迫骨折,NaN,単純
18761,89,女,心外,S5,全脊椎(単純)Sagのみ,亜急性に進行する下肢運動障害あり。 頸椎～胸椎～腰椎の腫瘍性病変や脊髄疾患精査目的です。,脊髄腫瘍,NaN,単純
18839,44,女,脳外,S6,脊椎(委任),全脊髄FIESTA axial (volumeでな くても良いから脂肪抑制) かならずaxi...,脳脊髄液漏出症の疑い,NaN,単純
19285,25,男,整形,U3,手関節,左TFCC損傷疑いです ECU腱は腱鞘から脱臼している状況です。,左ＴＦＣＣ損傷,NaN,単純


In [102]:
df = df.dropna()

In [103]:
df.loc[df[2].str.contains('総診'), 2] = '救急'
df.loc[df[2].str.contains('ﾍﾟｲﾝ'), 2] = '麻酔'

In [104]:
del df[7]

In [7]:
df = one_hot(df, 'sex')

['男' '女']


In [8]:
df = one_hot(df, 'CE')

['単純' '造影']


In [105]:
df[3] = df[3].replace('S7', 'S6')
df[3] = df[3].replace('LI4', 'LI3')

In [106]:
df[3].unique()

array(['A1', 'A2', 'AB1', 'AB3', 'AB4', 'AB5', 'AB6', 'B1', 'B2', 'BL1',
       'C1', 'C2', 'C3', 'DW', 'H1', 'H10', 'H12', 'H13', 'N1', 'H15',
       'H16', 'H17', 'H18', 'H19', 'H2', 'H20', 'H3', 'H4', 'H5', 'H6',
       'H7', 'H8', 'H9', 'HS', 'K1', 'K2', 'L1', 'L2', 'L3', 'L4', 'L5',
       'LI1', 'LI2', 'LI3', 'P1', 'PL1', 'R1', 'R2', 'S1', 'S2', 'S3',
       'S4', 'S5', 'S6', 'S9', 'U1', 'U2', 'U3', 'U4', 'U5'], dtype=object)

In [107]:
df[2].unique()

array(['脳内', '心外', '腎内', '皮膚', '循内', '内泌', '形外', '小児', '消外', '整形', '消内',
       '腫内', '放科', '泌尿', '救急', '母女', '乳一', '耳鼻', '精神', '呼外', '血内', '呼内',
       '口外', '脳外', '眼科', '麻酔'], dtype=object)

In [108]:
df[4].unique()

array(['胸部委任', '下肢血管：動脈（単純）', '下腿部', '下腹部(一般)', '腰椎(髄)', '骨盤部',
       '上腹部(肝胆膵他）', '上腹部委任', '腎臓', '婦人科下腹部', '泌尿器系下腹部', '胎児撮影(AFG)',
       '腸管(経口法)', 'MRdevice骨盤部', 'MRdevice腹部', '下肢(委任)', '眼窩', '胸部(郭)',
       '頚部', '頚部(委任)', '肩関節', '肩軟部', '股関節', '上下顎部', '上肢(委任)', '上腹部血管(造影）',
       '上腕部', '脊椎(委任)', '前腕部', '足関節', '足及び足指', '大腿部', '頭部(委任)', '頭部(脳)',
       '内耳(後頭蓋窩)', '乳房', '副鼻腔', '膝関節', '手関節', '手及び手指', '手首軟部', '足首軟部',
       '膝軟部', '肘関節', '肘軟部', '膀胱', '縦隔', 'MRdevice胸部', '心臓', '乳房Dynamic',
       'WB MRI', '全脊椎(単純)Sagのみ', 'MRA頚部', 'MRA脳', 'MRdevice頭頸部', '顎関節',
       '頭部,頚部(他)', '下垂体', '下垂体Dynamic', '全脊椎(造影)Sagのみ', '頚椎(髄)',
       '上腹部血管(単純）', '腎動脈（単純)', '副腎', '下腹部血管(単純）', 'MRdevice下肢',
       '上肢血管（造影）', '下肢血管：動脈（造影）', '胆道系(MRCP)', '前立腺', '下腹部委任', '直腸',
       '尿路系MR[VCG(単純)]', '胸椎(髄)', 'MRdevice上肢'], dtype=object)

In [109]:
df=df.rename(columns={0:'year',1:'sex',2:'section',3:'label',4:'position',5:'purpose',6:'diagnosis',8:'CE'})

In [110]:
df

,year,sex,section,label,position,purpose,diagnosis,CE
0,46,男,脳内,A1,胸部委任,脳梗塞原因検査中の方です。 恐縮ですが、大動脈プラークイメージおよび、頚部動脈プラークイメー...,アテローム血栓性脳梗塞・急性期,単純
1,72,男,脳内,A1,胸部委任,動脈硬化リスクの高い方。 無症候性脳梗塞の繰り返しのためAorta imag eも含め、弓部...,冠動脈バイパス術後 左鎖骨下動脈狭窄症,単純
2,72,女,心外,A1,胸部委任,術前精査,鎖骨下動脈瘤,単純
4,48,男,腎内,A2,下肢血管：動脈（単純）,糖尿病、血液透析中の患者さん。 両下肢の歩行時痺れ痛み。間欠性跛行あり。 ASOの精査目的,下肢閉塞性動脈硬化症,単純
5,60,女,皮膚,A2,下肢血管：動脈（単純）,上記で加療中です。足趾潰瘍と色調不良の悪化あります。血管の 状態悪化ありますでしょうか。,四肢バージャー病,単純
...,...,...,...,...,...,...,...,...
19331,64,女,整形,U5,肘関節,右肘痛精査目的,関節リウマチ・肘関節,単純
19332,67,女,整形,U5,肘関節,1ヶ月ほど前～左肘内側のしこり、圧痛あり CTで1cm程度の腫瘤？あります,関節リウマチ,単純
19333,73,男,整形,U5,肘関節,左肘部管症候群疑い 尺骨神経精査,左肘部管症候群,単純
19334,73,男,整形,U5,肘関節,肘部管精査目的 左尺骨神経障害,左肘部管症候群,単純


In [111]:
position_list = df['position'].unique()

df['position']=df['position'].replace( list(filter(lambda x: '下肢血管' in x,position_list)), '下肢血管')
df['position']=df['position'].replace( list(filter(lambda x: '頚部' in x,position_list)), '頚部')
df['position']=df['position'].replace( list(filter(lambda x: '上肢' in x,position_list)), '上肢')
df['position']=df['position'].replace( list(filter(lambda x: 'MRdevice' in x,position_list)), 'MRdevice')
df['position']=df['position'].replace( list(filter(lambda x: '全脊椎' in x,position_list)), '全脊椎')
df['position']=df['position'].replace( list(filter(lambda x: '腹部血管' in x,position_list)), '腹部血管')
df['position']=df['position'].replace( list(filter(lambda x: '下垂体' in x,position_list)), '下垂体')
df['position']=df['position'].replace( list(filter(lambda x: '乳房' in x,position_list)), '乳房')
df['position']=df['position'].replace( list(filter(lambda x: '乳房' in x,position_list)), '乳房')



df['position']=df['position'].replace( ['縦隔','胸部(郭)'], '胸郭')
df['position']=df['position'].replace( ['尿路系MR[VCG(単純)]', '泌尿器系下腹部'], '泌尿器系下腹部')
df['position']=df['position'].replace( ['下腹部委任', '下腹部(一般)'], '下腹部')

In [112]:
#条件にマッチしたIndexを取得
drop_index = df.index[(df['position'] == 'MRdevice') | 
                      (df['position'] == 'WB MRI') | 
                      (df['label'] == 'DW') 
                     ]
#drop_index + df.index[df['position'] == 'WB MRI']

print(drop_index)
#条件にマッチしたIndexを削除
df = df.drop(drop_index)

Int64Index([ 1371,  1372,  2589,  2887,  2888,  2953,  2955,  2957,  4713,
             4715,  4716,  4717,  4718,  4719,  4720,  4721,  4723,  4724,
             4725,  4726,  8156, 10046, 10047, 10048, 10049, 10050, 10051,
            10052, 10053, 10054, 10055, 10056, 10057, 10058, 10059, 10060,
            11735, 12277, 12531, 12776, 12777, 13803, 13804, 13805, 13806,
            17354, 17492, 17493, 17494, 17495, 18396],
           dtype='int64')


In [113]:
CE_list = df['CE'].unique()
df['CE']=df['CE'].replace( list(filter(lambda x: '造影' in x,CE_list)), '造影')

In [114]:
df['position'].value_counts()

頭部(脳)        4473
頭部(委任)       3472
胆道系(MRCP)    1542
婦人科下腹部       1159
腰椎(髄)         976
上腹部(肝胆膵他）     747
下垂体           554
頚椎(髄)         529
膝関節           452
MRA脳          390
全脊椎           382
前立腺           357
骨盤部           356
肩関節           353
頚部            287
大腿部           265
乳房            240
内耳(後頭蓋窩)      200
上下顎部          185
胸椎(髄)         166
眼窩            122
股関節           110
胸郭            108
副鼻腔           104
泌尿器系下腹部       101
腎臓            100
上腹部委任          98
膀胱             88
下腹部            83
手及び手指          69
足及び足指          65
上腕部            64
胸部委任           56
下腿部            52
心臓             45
手関節            43
顎関節            43
足関節            40
脊椎(委任)         38
胎児撮影(AFG)      33
肩軟部            28
前腕部            28
膝軟部            27
腸管(経口法)        25
腎動脈（単純)        25
肘関節            23
副腎             20
下肢血管           19
下肢(委任)         19
手首軟部           14
上肢             13
肘軟部            11
腹部血管           10
足首軟部            8
直腸              6
Name: posi

In [115]:
df['label'].value_counts()

H2     2785
H7     1627
LI2    1598
H6     1092
S4      894
AB4     776
H5      762
B1      732
H1      638
H8      587
LI3     522
N1      491
S1      476
P1      403
L2      401
AB3     371
S5      359
U2      333
H4      324
H10     323
B2      261
H19     255
C3      242
H9      221
LI1     203
H3      195
S6      167
R1      138
S3      137
K1      136
H13     121
C1      109
BL1     106
H18     100
AB1      89
L3       80
AB5      70
L1       69
H15      64
R2       60
H12      57
H17      49
C2       45
S2       44
PL1      41
K2       39
L4       36
U3       34
HS       30
L5       26
AB6      25
A2       19
H20      18
U5       10
S9       10
U4        9
H16       6
U1        5
A1        3
Name: label, dtype: int64

In [116]:
df['label'] = df['label'].replace({'H14':'N1'})

In [117]:
df['label'].unique()

array(['A1', 'A2', 'AB1', 'AB3', 'AB4', 'AB5', 'AB6', 'B1', 'B2', 'BL1',
       'C1', 'C2', 'C3', 'H1', 'H10', 'H12', 'H13', 'N1', 'H15', 'H16',
       'H17', 'H18', 'H19', 'H2', 'H20', 'H3', 'H4', 'H5', 'H6', 'H7',
       'H8', 'H9', 'HS', 'K1', 'K2', 'L1', 'L2', 'L3', 'L4', 'L5', 'LI1',
       'LI2', 'LI3', 'P1', 'PL1', 'R1', 'R2', 'S1', 'S2', 'S3', 'S4',
       'S5', 'S6', 'S9', 'U1', 'U2', 'U3', 'U4', 'U5'], dtype=object)

In [118]:
df['section'].value_counts()

脳外    4416
整形    2548
脳内    2205
消内    1928
母女    1197
小児     729
泌尿     655
呼内     654
消外     565
耳鼻     564
呼外     542
放科     461
皮膚     381
乳一     327
精神     273
口外     213
循内     205
眼科     181
血内     173
救急     146
内泌     145
心外      99
腎内      72
麻酔      69
形外      47
腫内      28
Name: section, dtype: int64

In [119]:
df.isnull().all(axis=0)

year         False
sex          False
section      False
label        False
position     False
purpose      False
diagnosis    False
CE           False
dtype: bool

In [120]:
df.to_csv("CorrectedData.csv",index=False)

In [121]:
df = pd.read_csv("CorrecedData.csv")

In [6]:
df =exam_preprosses(df)

In [8]:
df = to_vec(df)

/workspace/MRI/2022_MRI_Sc/MRI_SC/PP.py:399: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_diagnosis'] = df['diagnosis'].copy().apply(meishi)


In [ ]:
df.to_csv("data/vector_data(RP).csv")

In [412]:
df = pd.read_csv("CorrecedData.csv")